In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.3 MB/s eta 0:00:00


In [3]:
pip install ttach

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import

필요한 라이브러리 import

In [4]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import timm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
import ttach as tta

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

IMG_SIZE:224, EPOCHS:20, LEARNING_RATE:3e-4, BATCH_SIZE:32, SEED:41

In [6]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':20,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

모든 시드값들 고정

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

모든 이미지 파일 불러와서 하나의 리스트 all_img_list 생성

In [8]:
all_img_list = glob.glob('drive/MyDrive/Doedal/train/*/*')

In [9]:
all_img_list

['drive/MyDrive/Doedal/train/걸레받이수정/111.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/11.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/175.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/178.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/20.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/87.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/28.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/72.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/94.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/26.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/164.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/71.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/239.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/209.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/159.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/57.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/235.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/98.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/174.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/118.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/86.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/198.png',

이미지의 경로를 가져와 하자 카테고리(label)를 만들어주기 위해 '/'를 기준으로 split

In [12]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[4])

In [13]:
df.head()

,img_path,label
0,drive/MyDrive/Doedal/train/걸레받이수정/111...,걸레받이수정
1,drive/MyDrive/Doedal/train/걸레받이수정/11.png,걸레받이수정
2,drive/MyDrive/Doedal/train/걸레받이수정/175...,걸레받이수정
3,drive/MyDrive/Doedal/train/걸레받이수정/178...,걸레받이수정
4,drive/MyDrive/Doedal/train/걸레받이수정/20.png,걸레받이수정


train 데이터를 train과 validation용 데이터로 나눠 줌. 7:3

In [14]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

한국어로 되어있는 하자 레이블값을 LabelEncoder를 사용하여 숫자로 바꿔 줌.

In [15]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

## CustomDataset

맨 앞에서 설정했던 hyperparameter값대로 설정해주는 것 같음

In [16]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [17]:
# OneOf 제거, HorizontalFlip,randomBrightnessContrast만 유지 
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
# #weighted sampling
# a_list = train['label'].value_counts()
# class_counts = train['label'].value_counts().to_list()
# class_weights = [1 / a_list[i] for i in range(len(class_counts))]
# num_samples = sum(class_counts)
# labels_lists = train['label'].to_list()
# weights = [class_weights[labels_lists[i]] for i in range(int(num_samples))]
# sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples), replacement=True)

In [18]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

efficientnet_b0를 backbone 모델로 두고 시도

In [19]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        #self.backbone = models.efficientnet_b0(pretrained=True)
        # 모델 수정됨
        self.backbone = timm.create_model('resnet14t', pretrained=True, num_classes = 1000)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha]*19)
        self.alpha[18] = 1-alpha
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

## Train

In [21]:
# from datetime import datetime, timezone, timedelta

# # 시간 고유값 
# kst = timezone(timedelta(hours=9))        
# train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")

# # 기록 경로
# RECORDER_DIR = os.path.join('results',train_serial)
# # 현재 시간 기준 폴더 생성
# os.makedirs(RECORDER_DIR, exist_ok=True)
# print(RECORDER_DIR)

In [22]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    #criterion = nn.CrossEntropyLoss().to(device)
    criterion = FocalLoss(gamma=2, alpha=0.25)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [23]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [24]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.30122] Val Loss : [0.17887] Val Weighted F1 Score : [0.66572]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [19], Train Loss : [0.00369] Val Loss : [0.20793] Val Weighted F1 Score : [0.74066]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [20], Train Loss : [0.00704] Val Loss : [0.20148] Val Weighted F1 Score : [0.76876]


## Inference

In [25]:
test = pd.read_csv('drive/MyDrive/Doedal/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: x.replace('./', 'drive/MyDrive/Doedal/', 1))

In [26]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=2)

In [27]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
# model = BaseModel()
# model.eval()
# optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

# infer_model = torch.load(os.path.join(RECORDER_DIR, "best-model.pt"))
# infer_model = torch.load(os.path.join('../results/20230515_132413', "best-model.pt"))

In [ ]:
# tta_transforms = tta.Compose(
#     [   
#         tta.HorizontalFlip(),
#         tta.Rotate90(angles=[0, 5, 355]),
#         tta.Multiply(factors=[0.9, 1, 1.1])
#     ]
# )

In [ ]:
# tta_model = tta.ClassificationTTAWrapper(infer_model, tta_transforms)

In [28]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [29]:
preds

array(['훼손', '오염', '훼손', '몰딩수정', '오염', '훼손',
       '오염', '오염', '훼손', '오염', '몰딩수정', '오타공',
       '걸레받이수정', '오염', '훼손', '훼손', '훼손',
       '걸레받이수정', '오염', '오염', '창틀,문틀수정', '훼손',
       '훼손', '터짐', '창틀,문틀수정', '훼손', '훼손', '오염',
       '훼손', '훼손', '오염', '훼손', '훼손', '오염', '터짐',
       '훼손', '훼손', '훼손', '오타공', '훼손', '오염', '오염',
       '걸레받이수정', '터짐', '훼손', '석고수정', '훼손',
       '훼손', '훼손', '훼손', '훼손', '훼손', '오염', '훼손',
       '훼손', '훼손', '훼손', '터짐', '오염', '오염', '오타공',
       '몰딩수정', '훼손', '오염', '훼손', '훼손', '오염',
       '오염', '훼손', '걸레받이수정', '오염', '오염', '훼손',
       '훼손', '훼손', '훼손', '오염', '오염', '훼손', '오염',
       '오염', '훼손', '오염', '훼손', '훼손', '훼손', '훼손',
       '훼손', '들뜸', '오염', '오염', '훼손', '훼손', '오염',
       '훼손', '훼손', '터짐', '오염', '몰딩수정', '오염', '피스',
       '훼손', '훼손', '터짐', '훼손', '훼손', '가구수정', '훼손',
       '훼손', '훼손', '훼손', '훼손', '훼손', '훼손', '오염',
       '오염', '훼손', '훼손', '꼬임', '석고수정', '훼손', '훼손',
       '훼손', '걸레받이수정', '훼손', '훼손', '훼손', '훼손',
       '훼손', '몰딩수정', '훼손', '훼손', '꼬임', '오염',


## Submission

In [30]:
submit = pd.read_csv('drive/MyDrive/Doedal/sample_submission.csv')

In [31]:
submit['label'] = preds

In [33]:
submit.head()

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염


In [34]:
submit['label']

0            훼손
1            오염
2            훼손
3      몰딩수정
4            오염
          ...     
787          훼손
788          훼손
789          오염
790          훼손
791          훼손
Name: label, Length: 792, dtype: object

In [35]:
import chardet

In [36]:
print(chardet.detect(submit['label'][0].encode()))

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [37]:
# 인코딩 오류 수정하는 코드(자음 모음 분리 합쳐줌)
import unicodedata

for i in range(len(submit['label'])):
  submit['label'][i] = unicodedata.normalize('NFC',submit['label'][i])

In [38]:
submit['label'][0]

'훼손'

In [39]:
print(chardet.detect(submit['label'][0].encode()))

{'encoding': 'utf-8', 'confidence': 0.7525, 'language': ''}


In [41]:
submit.to_csv('drive/MyDrive/Doedal/resnet_submit_1.csv', index=False)